In [4]:
!pip install beautifulsoup4

Looking in indexes: https://artifactory.michelin.com/api/pypi/pypi/simple

   ---------------------------------------- 0/2 [soupsieve]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   ---------------------------------------- 2/2 [beautifulsoup4]



In [ ]:
# import os
# import requests
# from bs4 import BeautifulSoup
# from urllib.parse import urljoin
# import urllib3

# # 🔕 Kill SSL warnings (since we’re ignoring SSL certs)
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# BASE_URL = "https://www.emsa.europa.eu/publications/download/8244/5502/23.html"
# DOWNLOAD_DIR = "data/emsa_reports"
# os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# def get_pdf_links():
#     response = requests.get(BASE_URL, verify=False)  # 👈 Don't forget this
#     soup = BeautifulSoup(response.content, "html.parser")
#     pdf_links = []

#     for a_tag in soup.find_all("a", href=True):
#         href = a_tag["href"]
#         if href.endswith(".pdf"):
#             full_url = urljoin(BASE_URL, href)
#             pdf_links.append(full_url)

#     return pdf_links

# def download_pdfs(pdf_links):
#     for link in pdf_links:
#         filename = os.path.join(DOWNLOAD_DIR, link.split("/")[-1])
#         if not os.path.exists(filename):
#             print(f"Downloading: {filename}")
#             response = requests.get(link, verify=False)  # 👈 And this
#             with open(filename, "wb") as f:
#                 f.write(response.content)

# # Fire it up
# pdf_links = get_pdf_links()
# print(f"Found {len(pdf_links)} PDFs.")
# download_pdfs(pdf_links)


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Found 0 PDFs.


In [ ]:
# import requests
# from bs4 import BeautifulSoup

# url = "https://www.emsa.europa.eu/publications/documents.html"
# response = requests.get(url, verify=False)
# soup = BeautifulSoup(response.content, "html.parser")

# # See if there's anything juicy in anchor tags
# for a_tag in soup.find_all("a", href=True):
#     href = a_tag["href"]
#     if "pdf" in href.lower():
#         print(href)


In [ ]:
# href

'/disclaimer.html'

In [ ]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

BASE_URL = "https://www.emsa.europa.eu/publications.html"
HEADERS = {"User-Agent": "Mozilla/5.0"}
VERIFY_SSL = False
def get_category_links():
    print("🔍 Fetching publication categories...")
    res = requests.get(BASE_URL, headers=HEADERS, verify=VERIFY_SSL)
    soup = BeautifulSoup(res.content, "html.parser")
    links = set()
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("/"):
            href = urljoin(BASE_URL, href)
        if "publications" in href and href != BASE_URL and href not in links:
            links.add(href)
    return list(links)
def get_article_links():
    res = requests.get(BASE_URL, headers=HEADERS, verify=VERIFY_SSL)
    soup = BeautifulSoup(res.content, "html.parser")

    article_links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if "/publications/item/" in href:
            full_url = urljoin(BASE_URL, href)
            article_links.append(full_url)

    return list(set(article_links))  # Deduplicate

article_links = get_article_links()
print(f"🔗 Found {len(article_links)} article links")


🔗 Found 8 article links


In [15]:
def get_pdf_download_links(article_links):
    download_links = []
    for link in article_links:
        try:
            res = requests.get(link, headers=HEADERS, verify=VERIFY_SSL)
            soup = BeautifulSoup(res.content, "html.parser")
            for a in soup.find_all("a", href=True):
                href = a["href"]
                if "/publications/download/" in href and href.endswith(".html"):
                    full_url = urljoin(link, href)
                    download_links.append(full_url)
        except Exception as e:
            print(f"❌ Failed on {link}: {e}")
    return download_links

pdf_links = get_pdf_download_links(article_links)
print(f"📄 Found {len(pdf_links)} disguised PDF download links")


📄 Found 11 disguised PDF download links


In [17]:
pdf_links

['https://www.emsa.europa.eu/publications/download/8176/5463/23.html',
 'https://www.emsa.europa.eu/publications/download/8132/5402/23.html',
 'https://www.emsa.europa.eu/publications/download/8244/5502/23.html',
 'https://www.emsa.europa.eu/publications/download/8189/4729/23.html',
 'https://www.emsa.europa.eu/publications/download/7727/5119/23.html',
 'https://www.emsa.europa.eu/publications/download/8027/5119/23.html',
 'https://www.emsa.europa.eu/publications/download/8028/5119/23.html',
 'https://www.emsa.europa.eu/publications/download/8029/5119/23.html',
 'https://www.emsa.europa.eu/publications/download/8137/5418/23.html',
 'https://www.emsa.europa.eu/publications/download/8213/5484/23.html',
 'https://www.emsa.europa.eu/publications/download/8197/5472/23.html']

In [18]:
import os

DOWNLOAD_DIR = "data/emsa_reports"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def download_pdfs(pdf_links):
    for url in pdf_links:
        filename = url.split("/")[-3].replace(".html", ".pdf")
        filepath = os.path.join(DOWNLOAD_DIR, filename)
        if os.path.exists(filepath):
            print(f"✅ Already downloaded: {filename}")
            continue

        print(f"⬇️ Downloading {filename} ...")
        try:
            r = requests.get(url, headers=HEADERS, stream=True, verify=VERIFY_SSL)
            if "application/pdf" in r.headers.get("Content-Type", ""):
                with open(filepath, "wb") as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"✅ Saved: {filename}")
            else:
                print(f"⚠️ Not a PDF: {url}")
        except Exception as e:
            print(f"❌ Error downloading {url}: {e}")

download_pdfs(pdf_links)


⬇️ Downloading 8176 ...
✅ Saved: 8176
⬇️ Downloading 8132 ...
✅ Saved: 8132
⬇️ Downloading 8244 ...
✅ Saved: 8244
⬇️ Downloading 8189 ...
✅ Saved: 8189
⬇️ Downloading 7727 ...
✅ Saved: 7727
⬇️ Downloading 8027 ...
✅ Saved: 8027
⬇️ Downloading 8028 ...
✅ Saved: 8028
⬇️ Downloading 8029 ...
✅ Saved: 8029
⬇️ Downloading 8137 ...
✅ Saved: 8137
⬇️ Downloading 8213 ...
✅ Saved: 8213
⬇️ Downloading 8197 ...
✅ Saved: 8197


In [20]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

# Constants
BASE_URL = "https://www.emsa.europa.eu/publications.html"
DOWNLOAD_PREFIX = "/publications/download"
DOWNLOAD_DIR = "data/emsa_pdfs"
HEADERS = {"User-Agent": "Mozilla/5.0"}
VERIFY_SSL = False  # EMSA got trust issues

# Make sure download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)


def sanitize_filename(name):
    return re.sub(r"[\\/*?\"<>|]", "_", name)[:80]


def find_pdf_links():
    """
    Crawl EMSA publication base page, and extract all direct PDF URLs via embedded iframe.
    """
    print("🔍 Scraping EMSA for embedded PDF links...")

    try:
        res = requests.get(BASE_URL, headers=HEADERS, verify=VERIFY_SSL)
        res.raise_for_status()
    except Exception as e:
        print(f"❌ Failed to fetch base URL: {e}")
        return []

    soup = BeautifulSoup(res.content, "html.parser")
    links = []

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith(DOWNLOAD_PREFIX) and href.endswith(".html"):
            page_url = urljoin(BASE_URL, href)
            title = sanitize_filename(a.get_text(strip=True) or "EMSA_Document")
            links.append((page_url, title))

    return links


def extract_pdf_from_page(page_url):
    """
    Visit intermediate HTML page and extract embedded PDF URL from iframe.
    """
    try:
        res = requests.get(page_url, headers=HEADERS, verify=VERIFY_SSL)
        soup = BeautifulSoup(res.content, "html.parser")
        iframe = soup.find("iframe")
        if iframe and iframe.get("src", "").endswith(".pdf"):
            return urljoin(page_url, iframe["src"])
    except Exception as e:
        print(f"⚠️ Error extracting PDF from {page_url}: {e}")
    return None


def download_pdf(pdf_url, title, index):
    """
    Actually download the PDF and save it with a readable, indexed name.
    """
    filename = f"{index:03d}_{title}.pdf"
    path = os.path.join(DOWNLOAD_DIR, filename)

    if os.path.exists(path):
        print(f"✅ Already downloaded: {filename}")
        return

    print(f"⬇️ Downloading: {filename}")
    try:
        res = requests.get(pdf_url, headers=HEADERS, stream=True, verify=VERIFY_SSL)
        res.raise_for_status()

        if "application/pdf" in res.headers.get("Content-Type", ""):
            with open(path, "wb") as f:
                for chunk in res.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"✅ Saved: {filename}")
        else:
            print(f"⚠️ Not a PDF file: {pdf_url}")
    except Exception as e:
        print(f"❌ Error downloading {filename}: {e}")


# 🚀 Run the pipeline
def main():
    pdf_pages = find_pdf_links()
    print(f"\n🔗 Found {len(pdf_pages)} potential PDF pages.\n")

    for i, (html_url, title) in enumerate(pdf_pages):
        pdf_url = extract_pdf_from_page(html_url)
        if pdf_url:
            download_pdf(pdf_url, title, i)
        else:
            print(f"🚫 No PDF found at: {html_url}")


if __name__ == "__main__":
    main()


🔍 Scraping EMSA for embedded PDF links...

🔗 Found 36 potential PDF pages.



Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


🚫 No PDF found at: https://www.emsa.europa.eu/publications/download/8244/5502/23.html


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


🚫 No PDF found at: https://www.emsa.europa.eu/publications/download/8213/5484/23.html


...

🚫 No PDF found at: https://www.emsa.europa.eu/publications/download/8027/5119/23.html


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


🚫 No PDF found at: https://www.emsa.europa.eu/publications/download/8028/5119/23.html


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


🚫 No PDF found at: https://www.emsa.europa.eu/publications/download/8029/5119/23.html


In [21]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

# Constants
BASE_URL = "https://www.emsa.europa.eu/publications.html"
DOWNLOAD_PREFIX = "/publications/download"
OUTPUT_DIR = "data/emsa_htmls"
HEADERS = {"User-Agent": "Mozilla/5.0"}
VERIFY_SSL = False

os.makedirs(OUTPUT_DIR, exist_ok=True)

def sanitize_filename(name):
    return re.sub(r"[\\/*?\"<>|]", "_", name)[:80]

def find_html_pages():
    """
    Find all publication pages that have document-like HTML.
    """
    print("🔍 Scraping EMSA index for publication links...")
    try:
        res = requests.get(BASE_URL, headers=HEADERS, verify=VERIFY_SSL)
        res.raise_for_status()
    except Exception as e:
        print(f"❌ Failed to load index page: {e}")
        return []

    soup = BeautifulSoup(res.content, "html.parser")
    links = []

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith(DOWNLOAD_PREFIX) and href.endswith(".html"):
            full_url = urljoin(BASE_URL, href)
            title = sanitize_filename(a.get_text(strip=True) or "EMSA_Document")
            links.append((full_url, title))

    return links


def download_html_content(url, title, index):
    """
    Download the HTML content of the publication page and save it.
    """
    filename = f"{index:03d}_{title}.html"
    path = os.path.join(OUTPUT_DIR, filename)

    if os.path.exists(path):
        print(f"✅ Already saved: {filename}")
        return

    try:
        print(f"⬇️ Saving: {filename}")
        res = requests.get(url, headers=HEADERS, verify=VERIFY_SSL)
        res.raise_for_status()
        with open(path, "wb") as f:
            f.write(res.content)
    except Exception as e:
        print(f"❌ Failed to save {url}: {e}")


def main():
    pages = find_html_pages()
    print(f"\n🔗 Found {len(pages)} document-like HTML pages.\n")

    for i, (url, title) in enumerate(pages):
        download_html_content(url, title, i)


if __name__ == "__main__":
    main()


🔍 Scraping EMSA index for publication links...

🔗 Found 36 document-like HTML pages.

⬇️ Saving: 000_EMSA CAAR2024.pdf.html
⬇️ Saving: 001_EMSA_FACTS_FIGURES_2024.pdf.html
⬇️ Saving: 002_Seafarers Statistics in the EU 2023 data report.pdf.html
⬇️ Saving: 003_AFVs Guidance 1.2 2025.pdf.html
⬇️ Saving: 004_ADER 2024.pdf.html
⬇️ Saving: 005_emsa ipa-enp newsletter issue 2.pdf.html
⬇️ Saving: 006_EMTER_F&F_2025_EN.pdf.html
⬇️ Saving: 007_EMTER_F&F_2025_BG.pdf.html
⬇️ Saving: 008_EMTER_F&F_2025_CS.pdf.html
⬇️ Saving: 009_EMTER_F&F_2025_DA.pdf.html
⬇️ Saving: 010_EMTER_F&F_2025_DE.pdf.html
⬇️ Saving: 011_EMTER_F&F_2025_EL.pdf.html
⬇️ Saving: 012_EMTER_F&F_2025_ES.pdf.html
⬇️ Saving: 013_EMTER_F&F_2025_ET.pdf.html
⬇️ Saving: 014_EMTER_F&F_2025_FI.pdf.html
⬇️ Saving: 015_EMTER_F&F_2025_FR.pdf.html
⬇️ Saving: 016_EMTER_F&F_2025_GA.pdf.html
⬇️ Saving: 017_EMTER_F&F_2025_HR.pdf.html
⬇️ Saving: 018_EMTER_F&F_2025_HU.pdf.html
⬇️ Saving: 019_EMTER_F&F_2025_IT.pdf.html
⬇️ Saving: 020_EMTER_F&F_2025_L

In [22]:
import mimetypes

def fix_file_extensions(directory):
    """
    Detect 'html' files that are actually PDFs and rename them accordingly.
    """
    print("\n🕵️ Detecting fake HTMLs (a.k.a. PDFs in disguise)...")

    for filename in os.listdir(directory):
        if filename.endswith(".html"):
            filepath = os.path.join(directory, filename)
            try:
                with open(filepath, "rb") as f:
                    header = f.read(5)

                if header.startswith(b"%PDF"):
                    new_filename = filename.replace(".html", ".pdf")
                    new_path = os.path.join(directory, new_filename)
                    os.rename(filepath, new_path)
                    print(f"📄 Renamed: {filename} → {new_filename}")
                else:
                    print(f"✅ HTML confirmed: {filename}")
            except Exception as e:
                print(f"⚠️ Error checking {filename}: {e}")


In [23]:
if __name__ == "__main__":
    main()
    fix_file_extensions(OUTPUT_DIR)


🔍 Scraping EMSA index for publication links...

🔗 Found 36 document-like HTML pages.

✅ Already saved: 000_EMSA CAAR2024.pdf.html
✅ Already saved: 001_EMSA_FACTS_FIGURES_2024.pdf.html
✅ Already saved: 002_Seafarers Statistics in the EU 2023 data report.pdf.html
✅ Already saved: 003_AFVs Guidance 1.2 2025.pdf.html
✅ Already saved: 004_ADER 2024.pdf.html
✅ Already saved: 005_emsa ipa-enp newsletter issue 2.pdf.html
✅ Already saved: 006_EMTER_F&F_2025_EN.pdf.html
✅ Already saved: 007_EMTER_F&F_2025_BG.pdf.html
✅ Already saved: 008_EMTER_F&F_2025_CS.pdf.html
✅ Already saved: 009_EMTER_F&F_2025_DA.pdf.html
✅ Already saved: 010_EMTER_F&F_2025_DE.pdf.html
✅ Already saved: 011_EMTER_F&F_2025_EL.pdf.html
✅ Already saved: 012_EMTER_F&F_2025_ES.pdf.html
✅ Already saved: 013_EMTER_F&F_2025_ET.pdf.html
✅ Already saved: 014_EMTER_F&F_2025_FI.pdf.html
✅ Already saved: 015_EMTER_F&F_2025_FR.pdf.html
✅ Already saved: 016_EMTER_F&F_2025_GA.pdf.html
✅ Already saved: 017_EMTER_F&F_2025_HR.pdf.html
✅ Alrea

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

# Constants
BASE_URL = "https://www.emsa.europa.eu/publications.html"
DOWNLOAD_PREFIX = "/publications/download"
OUTPUT_DIR = "data/emsa_texts"
HEADERS = {"User-Agent": "Mozilla/5.0"}
VERIFY_SSL = False

os.makedirs(OUTPUT_DIR, exist_ok=True)

def sanitize_filename(name):
    return re.sub(r"[\\/*?\"<>|]", "_", name)[:80]

def find_html_pages():
    print("🔍 Scraping EMSA index for publication links...")
    try:
        res = requests.get(BASE_URL, headers=HEADERS, verify=VERIFY_SSL)
        res.raise_for_status()
    except Exception as e:
        print(f"❌ Failed to load index page: {e}")
        return []

    soup = BeautifulSoup(res.content, "html.parser")
    links = []

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith(DOWNLOAD_PREFIX) and href.endswith(".html"):
            full_url = urljoin(BASE_URL, href)
            title = sanitize_filename(a.get_text(strip=True) or "EMSA_Document")
            links.append((full_url, title))

    return links

def extract_and_save_text(url, title, index):
    filename = f"{index:03d}_{title}.txt"
    path = os.path.join(OUTPUT_DIR, filename)

    if os.path.exists(path):
        print(f"✅ Already extracted: {filename}")
        return

    try:
        print(f"📝 Extracting: {filename}")
        res = requests.get(url, headers=HEADERS, verify=VERIFY_SSL)
        res.raise_for_status()
        soup = BeautifulSoup(res.content, "html.parser")

        # Extract text from all relevant tags
        content = []
        for tag in soup.find_all(['h1', 'h2', 'h3', 'p', 'li']):
            text = tag.get_text(strip=True)
            if text:
                content.append(text)

        full_text = "\n".join(content)

        with open(path, "w", encoding="utf-8") as f:
            f.write(full_text)

    except Exception as e:
        print(f"❌ Failed to extract from {url}: {e}")

def main():
    pages = find_html_pages()
    print(f"\n🔗 Found {len(pages)} document-like HTML pages.\n")

    for i, (url, title) in enumerate(pages):
        extract_and_save_text(url, title, i)

if __name__ == "__main__":
    main()


🔍 Scraping EMSA index for publication links...


c:\BUSDATA\envs\py310\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.emsa.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



🔗 Found 33 document-like HTML pages.

📝 Extracting: 000_EMSA Catalogue 2025 v26.06.pdf.txt


c:\BUSDATA\envs\py310\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.emsa.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


📝 Extracting: 001_EMSA CAAR2024.pdf.txt


...

c:\BUSDATA\envs\py310\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.emsa.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


📝 Extracting: 031_European Maritime Transport Environmental Report 2025.pdf.txt


c:\BUSDATA\envs\py310\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.emsa.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


📝 Extracting: 032_EMSA Outlook 2025_v3.0.pdf.txt


c:\BUSDATA\envs\py310\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.emsa.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
